In [1]:
import numpy as np
import random
import math

In [2]:
def LineGenerate(lenght, m):
    """
    Creating a line with m random segments
    """
    line = np.zeros(lenght,)
    ab = np.array(sorted(random.sample(range(0, lenght-1), m*2))) # random start and end of the segments
    ab = ab.reshape((m,2)) # start and end of the segments coordinates
    # positive points coordinates
    ones = np.arange(ab[0,0],ab[0,1])
    for i in range(1,m):
        ones = np.concatenate((ones, np.arange(ab[i,0],ab[i,1])))
    line[ones] = 1  # put the generated segments on the line
    return line, ones, ab

In [3]:
# generate a line to cover
line, ones, ab = LineGenerate(350, 15)

In [4]:
base = np.array((0,500)) # base station coordinates
L = 1275 # maximum tour length

In [5]:
# right side only
r_line = line[base[0]:]
r_ones=ones[np.argmax(ones>=base[0]):]-base[0]

In [6]:
def FindAssociated(c, ab, base, L):
    # distance from the farthest point to the base
    b = math.sqrt(c**2+base[1]**2)
    # distance from the farthest point to greedy
    dist = (L**2-2*L*b)/(2*(L-b-c))
    c_ass = c-dist
    for i in range(0,ab.shape[0]):
        if (c_ass > ab[i-1,1]) & (c_ass < ab[i,0]):
            c_ass = ab[i,0]
        if (c_ass < ab[0,0]):
            c_ass = ab[0,0]  
    return c_ass

In [7]:
def Candidates(base, L, ab):
    on_gap = False
    xB = ab[ab.shape[0]-1,1]
    c = np.array(xB)
    c_ass = np.array(FindAssociated(xB, ab, base, L))
    while not on_gap:
        # distance from the farthest point to the base
        b = math.sqrt(xB**2+base[1]**2)
        # distance from the farthest point to greedy
        dist = (L**2-2*L*b)/(2*(L-b-xB))
        # greedy point coordinate
        xA = xB-dist
        ass = FindAssociated(xA, ab, base, L)  
        c = np.append(c, xA)
        c_ass = np.append(c_ass, ass)
        xB = xA
        for i in range(0,ab.shape[0]):
            if (xA > ab[i-1,1]) & (xA < ab[i,0]):
                on_gap = True 
            if (xA < ab[0,0]):
                on_gap = True 
    c = c[:c.size-1]
    c_ass = c_ass[:c_ass.size-1]
    return c, c_ass

In [8]:
c_i, c_ass_i = Candidates(base, L, ab)
c_ass_i

array([221.00701596,  31.        ])

In [9]:
def AllCandidates(base, L, ab):
    ab_cut = ab
    c = np.empty(0)
    c_ass = np.empty(0)
    while not (ab_cut.size == 0):
        c_i, c_ass_i = Candidates(base, L, ab_cut)
        c = np.append(c, c_i)
        c_ass = np.append(c_ass, c_ass_i)       
        ab_cut = ab_cut[:ab_cut.shape[0]-1]
    return c, c_ass

In [10]:
c, c_ass = AllCandidates(base, L, ab)

In [11]:
def SegmentNumber(c, ab):
    for i in range(0,ab.shape[0]):
        if (c >= ab[i,0]) & (c <= ab[i,1]):
            return i  

In [12]:
def Recursion(c_k, ab, base, L):
    
    c_ass = FindAssociated(c_k, ab, base, L)
    j = np.arange(SegmentNumber(c_ass, ab), SegmentNumber(c_k, ab)+1)   
    if c_ass == ab[0,0]:
        return (c_k - ab[0,0])
    
    elif c_ass in ab[1:,0]:
        func = np.empty(0)
        for i in j:
            func = np.append(func, (c_k - ab[i,0]) + Recursion(ab[i-1,1], ab, base, L))    
        #func1 = (c_k - ab[j,0]) + func
        print(func)
        return min(func)
    
    else: 
        func = np.empty(0)
        for i in j:
            func = np.append(func, (c_k - ab[i,0]) + Recursion(ab[i-1,1], ab, base, L))    
        #func1 = (c_k - ab[j,0]) + func
        
        func2 = L + Recursion(c_ass, ab, base, L)
        
        return min(func2, min(func))

In [13]:
def DynamicProgramming(c, ab, base, L):
    c = np.sort(c)
    E = np.zeros(c.size) # check if zeros are ok
    

    for k in range(0, c.size):
        c_ass = FindAssociated(c[k], ab, base, L)
    
        j = np.arange(SegmentNumber(c_ass, ab), SegmentNumber(c[k], ab)+1)  
    
        if c_ass == ab[0,0]:
            E[k] = (c[k] - ab[0,0])
            
        elif c_ass in ab[1:,0]:
            func = np.empty(0)
            for i in j:
                func = np.append(func, (c[k] - ab[i,0]) + E[c.tolist().index(ab[i-1,1])])
            E[k] = min(func)
    
        else: 
            func = np.empty(0)
            for i in j:
                func = np.append(func, (c[k] - ab[i,0]) + E[c.tolist().index(ab[i-1,1])])      
            func2 = L + E[c.tolist().index(c_ass)]
            E[k] = min(func2, min(func))
            
    return E[c.size-1]

In [14]:
# generate a line to cover
line, ones, ab = LineGenerate(350, 15)
base = np.array((100,500)) # base station coordinates

In [15]:
ab_right = ab[np.argmax(ab[:,0]>base[0]):]-base[0]
ab_left =np.flip(base[0] - ab[:np.argmax(ab[:,0]>base[0])-1])

In [16]:
L = 1120
c_right, _ = AllCandidates(base, L, ab_right)
c_left, _ = AllCandidates(base, L, ab_left)

c_right = np.sort(c_right)
c_left = np.sort(c_left)

In [17]:
Dyn_right = DynamicProgramming(c_right, ab_right, base, L)
Dyn_right

170.0

In [18]:
Dyn_left = DynamicProgramming(c_left, ab_left, base, L)
Dyn_left 

42.0

In [19]:
def FindCentralTours(c_left, c_right , base, L):
    j = 0
    central_tours = np.empty((0,3))
    tour_lenght = 0
    while (tour_lenght <= L) and (j < c_left.size):
        central_tours_1 = np.empty((0,3))
        i = 0
        while (tour_lenght <= L) and (i < c_right.size):
            dist_left = math.sqrt(c_left[j]**2+base[1]**2)
            dist_right = math.sqrt(c_right[i]**2+base[1]**2)
            tour_lenght = c_left[j] + c_right[i] + dist_left + dist_right
            tour = np.array((c_left[j], c_right[i], tour_lenght)).reshape(1,3)
            central_tours_1 = np.append(central_tours_1, tour, axis=0)
            i += 1
        central_tours = np.append(central_tours, central_tours_1[central_tours_1[:,2]<=L,:], axis=0)
        tour_lenght = 0
        j += 1
    return central_tours

In [20]:
central_tours = FindCentralTours(c_left, c_right , base, L)

In [21]:
c_left_first = np.unique(np.sort(central_tours[:,0]))
c_right_first = np.unique(np.sort(central_tours[:,1]))

In [30]:
E_left = np.zeros(c_left_first.size)
ab_left_i = ab_left.astype(np.float32)
for i in range(0, c_left_first.size):
    if c_left_first[i] != ab_left_i[0,1]:
        ab_left_i[0,0] = c_left_first[i]
    else:
        ab_left_i = ab_left[ab_left[:,0]>c_left_first[i]]
    E_left[i] = DynamicProgramming(c_left[c_left>c_left_first[i]], ab_left_i, base, L)

[[67 69]
 [90 98]]
[[90 98]]


In [32]:
E_right = np.zeros(c_right_first.size)
ab_right_i = ab_right.astype(np.float32)
for i in range(0, c_right_first.size):
    if c_right_first[i] != ab_right_i[0,1]:
        ab_right_i[0,0] = c_right_first[i]
    else:
        ab_right_i = ab_right[ab_right[:,0]>c_right_first[i]]
    E_right[i] = DynamicProgramming(c_right[c_right>c_right_first[i]], ab_right_i, base, L)

[[ 39  61]
 [ 70  85]
 [ 87  94]
 [ 95 105]
 [107 129]
 [143 148]
 [153 161]
 [163 166]
 [181 229]
 [234 240]]


In [47]:
left_central = np.append(c_left_first.reshape(E_left.shape[0],1), E_left.reshape(E_left.shape[0],1), axis=1)

In [48]:
right_central = np.append(c_right_first.reshape(E_right.shape[0],1), E_right.reshape(E_right.shape[0],1), axis=1)

In [131]:
totalL = np.zeros(central_tours.shape[0])
for i in range (0,central_tours.shape[0]):
    a = tuple(np.argwhere(left_central == central_tours[i,0])[0])
    b = tuple(np.argwhere(right_central == central_tours[i,1])[0])
    totalL[i] = left_central[a[0],i] + right_central[b[0],i] + float(central_tours[i,2])